### Circle-Inspired Optimization Algorithm (CIOA)

Implemented in Python

Developed by: Otávio Augusto Peter de Souza (1) and Letícia Fleck Fadel Miguel (2)

(1) otavio.souza@ufrgs.br, (2) letffm@ufrgs.br

Federal University of Rio Grande do Sul (UFRGS)<br> Postgraduate Program in Civil Engineering (PPGEC)

https://doi.org/10.1016/j.softx.2022.101192

Paper Published in: SoftwareX<br>de Souza, O.A.P., and Miguel, L.F.F. (2022). CIOA: Circle-Inspired Optimization Algorithm, an Algorithm for Engineering Optimization. SoftwareX, Volume 19, July 2022, 101192. https://doi.org/10.1016/j.softx.2022.101192

Version: V01, Updated in: September 2022

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

In [ ]:
begin = time.time()

#Define in this cell the details of your objective function

def ObjectiveFunctionCIOA (x):
    
    
    ### OBJECTIVE FUNCTION ###

    k1 = 0.09755988
    k2 = 0.99*k1
    k3 = 0.0391908
    k4 = 0.9*k3
    
        
    obj = -x[3]           # Objective Function
    
    
    ### CONSTRAINTS ###
    
    nug = 1               # Number of Inequality Constraints - Use 0 if your problem has no inequality constraints.
    nuh = 4               # Numer of Equality Constraints - Use 0 if your problem has no equality constraints.
    
    rg = np.zeros(nug)
    rh = np.zeros(nuh)
    
    # Inequality Constraints:
    
    rg[0] = x[4]**0.5+x[5]**0.5-4;        # Enter your inequality equations.
    
    
    # Equality Constraints:
    
    rh[0] = x[0]+k1*x[1]*x[4]-1;          # Enter your equality equations.
    rh[1] = x[1]-x[0]+k2*x[1]*x[5];
    rh[2] = x[2]+x[0]+k3*x[2]*x[4]-1;
    rh[3] = x[3]-x[2]+x[1]-x[0]+k4*x[3]*x[5];
    
    
    ### PENALTIES ###
    
    Weight = 10**5                       #Define a value multiplied by the penalty
    
    Pen = 0
    for icong in range(nug):
        Pen = Pen+(max(rg[icong],0))                # Penalty for inequality constraints
    
    for iconh in range(nuh):
        Pen = Pen+(max(abs(rh[iconh])-0.0001,0))    # Penalty for equality constraints
    
    Pen = Weight*Pen
    
    sol = obj+Pen         # Objective function penalized
    
    return sol, obj, rg, rh, nug, nuh

In [ ]:
### INPUT DATA ###

Nvar = 6                                         # Number of variables of the objective function
Nag = 250                                        # Number of search agents
Nit = 400                                        # Number of iterations
Ub = np.array([1, 1, 1, 1, 16, 16])              # Upper bound of the design variables
Lb = np.array([0, 0, 0, 0, 0.00001, 0.00001])    # Lower bound of the design variables

ThetaAngle = 17                                  # Theta angle in degrees (suggestions: 13, 17 or 19°)
GlobIt = 0.85                                    # Proportion of iterations for global search (sugg.: 0.75 to 0.95)


In [ ]:
### ALGORITHM  INITIALIZATION ###

Angle = (ThetaAngle*np.pi/180)

Convergence = np.zeros(Nit)
BestVar = np.zeros((Nit,Nvar))                   # Initializing solution vectors
x = np.ones(Nvar)
Bestobj = np.zeros((Nit,1))

In [ ]:
### RADIUS CALCULATION ###

raux = 1*(np.sqrt(max(Ub)-min(Lb)))/Nag          # Radius auxiliary variable
r = np.zeros(Nag)                                # Initializing radius vector

h = Nag
while h >= 1:
    r[h-1] = raux*(h)**2/Nag                     # Completing radius vector
    h = h-1

In [ ]:
### RANDOM SOLUTION DETERMINATION ###

Var0 = np.zeros((Nag,Nvar))
obj0 = np.zeros((Nag,1))


for i in range(Nag):
    for j in range(Nvar):
        Var0[i,j] = Lb[j]+(Ub[j]-Lb[j])*random.random()    # Random variables generation


Sol0 = np.zeros(Nag)

for i in range(Nag):
    x = Var0[i,:]                                            # Assigns the random variables to the design variables
    sol, obj, rg, rh, nug, nuh = ObjectiveFunctionCIOA(x)    # Calls the objective function
    Sol0[i] = sol                                            # Random solutions for each search agent


In [ ]:
### GLOBAL SEARCH LOOP ###

It = 0

while It <= round(Nit*GlobIt):
    order = np.argsort(Sol0)           # Solution vector classification
    for i in range(Nag):
        for j in range(Nvar):          # Update of the position of agents in the global search
            if (j%2) == 0:
                Var0[i,j] = (Var0[order[i],j]) - ((random.random())*r[i]*np.cos((It*Angle)-Angle)) + ((random.random())*r[i]*np.cos(It*Angle))
            else:
                Var0[i,j] = (Var0[order[i],j]) - ((random.random())*r[i]*np.sin((It*Angle)-Angle)) + ((random.random())*r[i]*np.sin(It*Angle))
    
    for i in range(Nag):               # Update of variables that exceeded the limits
        for j in range (Nvar):
            if Var0[0,j] > Ub[j]:
                Var0[0,j] = Ub[j]
            elif Var0[0,j] < Lb[j]:
                Var0[0,j] = Lb[j]
            
            if Var0[i,j] > Ub[j]:
                Var0[i,j] = Var0[0,j]
            elif Var0[i,j] < Lb[j]:
                Var0[i,j] = Var0[0,j]
    
    for i in range(Nag):
        x = Var0[i,:]                                          # Assigns the new particle positions to the design variables
        sol, obj, rg, rh, nug, nuh = ObjectiveFunctionCIOA(x)  # Calls the objective function
        Sol0[i] = sol                                          # Solutions according to the new positions for each search agent
        obj0[i] = obj
    
    
    Position = np.argmin(Sol0)            # Finding the best solution among agents
    BestSolution = Sol0[Position]       # Saves the best solution of the iteration
    BestVar[It,:] = Var0[Position,:]    # Variables corresponding to the best solution
    Bestobj[It] = obj0[Position]
    Convergence[It] = BestSolution
    
    if (It%(round(360/ThetaAngle))) == 0:
        for i in range(Nag):
            r[i] = r[i]*0.99               # Radius reduction every 360º (full lap) of the algorithm
        
    It = It+1

In [ ]:
### LOCAL SEARCH INITIALIZATION ###

Lb1 = np.zeros(Nvar)
Ub1 = np.zeros(Nvar)

for i in range(Nvar):
    Lb1[i] = BestVar[It-1,i]-((Ub[i]-Lb[i])/10000)      # Reducing the limits of the design variables
    Ub1[i] = BestVar[It-1,i]+((Ub[i]-Lb[i])/10000)

Var1 = np.zeros((Nag,Nvar))
for i in range(Nag):
    Var1[i,:] = BestVar[It-1,:]                         # All search agents will begin their search in the promising region

x = Var1[1,:]
sol, obj, rg, rh, nug, nuh = ObjectiveFunctionCIOA(x)   # Calls the objective function
Sol1 = np.zeros(Nag)

for i in range(Nag):
    Sol1[i] = sol



In [ ]:
### LOCAL SEARCH LOOP ###

while It <= Nit-1:
    order = np.argsort(Sol1)     # Solution vector classification
    for i in range(Nag):
        for j in range(Nvar):
            if (j%2) == 0:
                Var1[i,j] = (Var1[order[i],j]) - ((random.random())*r[i]*np.cos((It*Angle)-Angle)) + ((random.random())*r[i]*np.cos(It*Angle))
            else:
                Var1[i,j] = (Var1[order[i],j]) - ((random.random())*r[i]*np.sin((It*Angle)-Angle)) + ((random.random())*r[i]*np.sin(It*Angle))
    
    for i in range(Nag):
        for j in range(Nvar):
            if Var1[0,j] > Ub[j]:
                Var1[0,j] = Ub[j]
            elif Var1[0,j] < Lb[j]:
                Var1[0,j] = Lb[j]
            
            if Var1[i,j] > Ub1[j]:
                Var1[i,j] = Var1[0,j]
            elif Var1[i,j] > Ub[j]:
                Var1[i,j] = Ub[j]
            
            if Var1[i,j] < Lb1[j]:
                Var1[i,j] = Var1[0,j]
            elif Var1[i,j] < Lb[j]:
                Var1[i,j] = Lb[j]
    
    for i in range(Nag):
        x = Var1[i,:]
        sol, obj, rg, rh, nug, nuh = ObjectiveFunctionCIOA(x)       # Calls the objective function
        Sol1[i] = sol
        obj0[i] = obj
    
    Position = np.argmin(Sol1)
    BestSolution = Sol1[Position]
    BestVar[It,:] = Var1[Position,:]
    Bestobj[It] = obj0[Position]
    Convergence[It] = BestSolution
    
    if (It%(round(360/ThetaAngle))) == 0:
        for i in range(Nag):
            r[i] = r[i]*0.99
        
    It = It+1    

In [ ]:
### CONVERGENCE CURVE AND RESULTS CONSTRUCTION ###

ConvergenceCurve = np.zeros(Nit)
ConvergencePlot = np.zeros(Nit)
ConvergenceCurve[0] = Convergence[0]
ConvergencePlot[0] = Bestobj[0]

iconv=1
while iconv <= Nit-1:
    if Convergence[iconv] < ConvergenceCurve[iconv-1]:
        ConvergenceCurve[iconv] = Convergence[iconv]
        ConvergencePlot[iconv] = Bestobj[iconv]
    else:
        ConvergenceCurve[iconv] = ConvergenceCurve[iconv-1]
        ConvergencePlot[iconv] = ConvergencePlot[iconv-1]
    
    iconv = iconv+1

PositionSol = np.argmin(Convergence)
BestSolution = ConvergencePlot[Nit-1]
BestVariables = BestVar[PositionSol,:]

In [ ]:
### CONSTRAINT VIOLATIONS ###

x = BestVariables

sol, obj, rg, rh, nug, nuh = ObjectiveFunctionCIOA(x)

Violg = 0
for icong in range(nug):                         # Violation of inequality constraints
    Violg = Violg+max(rg[icong],0)

Violh = 0
for iconh in range(nuh):                         # Violation of equality constraints
    Violh = Violh+(max(abs(rh[iconh])-0.0001,0))
    
if (nug+nuh) == 0:                               # Total violation
    Violation = (Violg+Violh)/1
else:
    Violation = (Violg+Violh)/(nug+nuh)

In [ ]:
### CONVERGENCE CURVE DISPLAY ###

plt.figure(1,figsize=(12,6))
plt.plot(ConvergencePlot,'r')
plt.xlabel('Iteration Number'); plt.ylabel('Objective Function Value');
plt.xlim(1,Nit);
plt.title('Convergence Curve');
plt.grid(True)

In [ ]:
### RESULTS DISPLAY ###

print('BestSolution:       ', BestSolution)         # Display of best solution
print('Best Variables:     ', BestVariables)        # Display of the design variables that constitute the best solution
print('Violation:          ', Violation)            # Display violation

end = time.time()

time = end - begin
print('Computational Time: ', time)                 # Display Computational Time

In [ ]:
# CITE AS:
# de Souza, O.A.P., and Miguel, L.F.F. CIOA: Circle-Inspired Optimization Algorithm, an algorithm for engineering optimization.
# SoftwareX 2022;19:101192. https://doi.org/10.1016/j.softx.2022.101192